# Probabilidad y estadística interactiva

## using

* ipywidgets
* seaborn

* voila
* reveal

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import expon
from scipy.stats import binom
from scipy.stats import uniform
import ipywidgets as widgets
from IPython.display import display
from IPython.display import display_html
from IPython.display import display_markdown
import seaborn as sns
import pandas as pd
import plotly.express as px
import ipywidgets
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

### Número de bines en un histograma

- No hay un número "mejor" de bines. Diferentes números de bines pueden revelar diferentes características de los datos.
- Aunque existen varias pautas y reglas generales para determinar un número "óptimo" de bines, estos métodos generalmente hacen fuertes suposiciones sobre la forma de la distribución. 
  - Según la distribución real de los datos y los objetivos del análisis, pueden ser apropiados diferentes anchos de intervalo, por lo que generalmente **se necesita experimentación para determinar un número de bines apropiado.** 
  
- Vaya a la diapositiva abajo y determine el número adecuado de bines. 

In [2]:
np.random.seed(123)
size = 100

# create some exponential random data
values = expon.rvs(scale=1/4, size=size)

# Create gaussian noise
noise = norm.rvs(loc=1, size=size)

real_data = values + np.abs(noise)

def hist_bins(b):
    plt.figure(figsize=(8,6))
    #plt.hist(real_data, bins=b, density=True, edgecolor="seagreen")
    ax = sns.histplot(x=real_data, bins=b, stat="density", kde=True)
    ax.lines[0].set_color('crimson')
    plt.xlabel("X")

b = widgets.SelectionSlider(options=list(range(1,11))+[20, 30, 50, 100], value=2,
    description='Número de bines',
    layout=widgets.Layout(width='70%'))
b.style = {'description_width': '300px'}

ui = widgets.VBox(children=[b])

out = widgets.interactive_output(hist_bins, {'b': b})

display(ui, out)

Output()

In [3]:
l = widgets.Label("¿El número adecuado de bines es?")

n = widgets.Dropdown(options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], value=1,
    description='Seleccione un número de bines',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

a = widgets.Label("")

ui = widgets.VBox(children=[l, a, n])

def answer(n):
    if n==4:
        print("Correcto!")
    elif n in [3, 6]:
        print("Podría ser!")
    else:
        print("Incorrecto!")

out = out = widgets.interactive_output(answer, {'n': n})
display(ui, out)

Output()

### Explorando la correlación

- A continuación se muestran ejemplos de diagramas de dispersión de varios conjuntos de datos con varios coeficientes de correlación.
- Vaya a la diapositiva abajo y seleccione el coeficiente de correlación entre -1 y 1. 
- También puede cambiar el tamaño de la muestra y observar la correlación de acuerdo al número de puntos observados. 

In [4]:
mean = [0, 0]

def cov_plot(c, n, l):
    np.random.seed(123456)
    cov = [[1, c], [c, 1]]
    x, y = np.random.multivariate_normal(mean, cov, n).T
    data = pd.DataFrame({'x': x, 'y': y})
    s = sns.lmplot(data=data, x='x', y='y', height=7, aspect=1, fit_reg=l,
                   scatter_kws={"s": 20, 'alpha': 0.5, 'color': "seagreen"},
                   line_kws={'color': 'orange'}
                  )

c = widgets.SelectionSlider(value=0, options=[-1.0, -0.9, -0.8, -0.7, -0.6, 
                                              -0.5, -0.4, -0.3, -0.2, -0.1] + 
                            [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                            description='Correlación',
                            layout=widgets.Layout(width='70%'))
c.style = {'description_width': '300px'}

n = widgets.SelectionSlider(value=100, 
                            options=list(range(10,101,10))+[200, 500, 1000, 5000],
                            description='Tamaño de muestra',
                            layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

l = widgets.Checkbox(
    value=True,
    description='Regresión lineal',
    layout=widgets.Layout(width='70%'))
l.style = {'description_width': '300px'}

ui = widgets.VBox(children=[c, n, l])

out = widgets.interactive_output(cov_plot, {'c':c, 'n': n, 'l': l})

display(ui, out)

Output()

### Función de masa y función acumulada de probabilidad

- Para una variable aleatoria discreta $X$ con valores posibles $x_1, x_2, x_3, \cdots , x_n$, una función de masa de probabilidad $f(x_i)$ es una función que cumple las siguientes condiciones:
    
  1. $f(x_i) \geq 0$
  2. $\sum^n_{i=1} f(x_i) = 1$
  3. $f(x_i) = P(X = x_i)$
  
- Para una variabel aleatoria discreta $X$, la Función de Distribución Acumulada $F(x)$ satisface las siguientes propiedades:
  1. $F(x)=P(X \leq x) = \sum_{x_i \leq x} f(x_i)$
  2. $0 \leq F(x) \leq 1$
  3. Si $x \leq y$, entonces $F(x) \leq F(y)$

In [5]:
def pmf_cdf(a, b, c):
    d = pd.DataFrame({"x": [-1, 0, 1, 4], "p(x)": ['0.2', '0.5', a, '0.1'],
                      "F(x)": [b, '0.7', c, '1.0'],
                     })
    
    d = d.reset_index()
    d = d.drop(['index'],axis=1)
    
    #d = d.style.set_table_styles([{'selector': 'th', 'props': 
    #                               [('font-size', '20pt')]}]).set_properties(**{'font-size': '20pt'})
        
    true_values = {"a": "0.2", "b": "0.2", "c": "0.9"}
    
    correct_values = {v: eval(v).value==true_values[v] for v in true_values}
    
    #print(', '.join(["("+tv+") Correcta" if correct_values[tv] 
    #                                          else "("+tv+") Incorrecta" 
    #           for i, tv in enumerate(correct_values)])+".")
    
    #dh = display_html(f"<center>{d.to_markdown()}</center>", raw=True)
    display_markdown(d.to_markdown(), raw=True)
    
    check(a, b, c)

l1 = widgets.Label(r"A partir de la tabla con la función de masa de probabilidad $p(x)$")
l2 = widgets.Label(r" y la función de distribución acumulada $F(x)$, ") 
l3 = widgets.Label(r"de los valores de $a$, $b$ y $c.$")

texto = "A partir de la tabla con la función de masa de probabilidad p(x) y la función de distribución acumulada F(x), de los valores de a, b y c."

l = widgets.HTML(value= '<p>'+ texto +' </p>')

lf = widgets.Label("")

a = widgets.Dropdown(options=['a', '0.0', '0.1', '0.2', '0.5', '0.8', '0.9', '1.0'], value='a',
    description='Valor de a',
    layout=widgets.Layout(width='70%'))
a.style = {'description_width': '300px'}

b = widgets.Dropdown(options=['b', '0.0', '0.1', '0.2', '0.5', '0.8', '0.9', '1.0'], value='b',
    description='Valor de b',
    layout=widgets.Layout(width='70%'))
b.style = {'description_width': '300px'}

c = widgets.Dropdown(options=['c', '0.0', '0.1', '0.2', '0.5', '0.8', '0.9', '1.0'], value='c',
    description='Valor de c',
    layout=widgets.Layout(width='70%'))
c.style = {'description_width': '300px'}

button = widgets.Button(description="Comprobar")

def check(a, b, c):
    texto = ""
    if a=='0.2': texto += "(a) correcta, "
    else: texto += "(a) incorrecta, "
    if b=='0.2': texto += "(b) correcta, "
    else: texto += "(b) incorrecta, "
    if c=='0.9': texto += "(c) correcta. "
    else: texto += "(c) incorrecta. "
    #texto += ' '.join([a, b, c])
    print(texto)

ui = widgets.VBox(children=[l, a, b, c])
out = widgets.interactive_output(pmf_cdf, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

### Lanzando monedas (distribución binomial)

- Para la siguiente distribución binomial, elija el número de monedas $n$ que desea lanzar
$X\sim binom(p=0.5, n)$. 
- El resultado cuenta el número de caras obtenidas al lanzar $n$ monedas. 

  - Por ejemplo al lanzar 2 monedas pueden obtenerse $x \in \{0, 1, 2\}$ caras. 

- Elija el número de lanzamientos que desea realizar, a medida que este número sea más grande, más cercano será el valor observado en la muestra al valor teórico (PMF). 

In [6]:
def coin(n, p, s):
    X = binom.rvs(n, p, size=s)
    x, freq = np.unique(X, return_counts=True)
    plt.figure(figsize=(8,6))
    plt.bar(x, freq/s, label="Simulación")
    plt.xlabel("Número de caras")
    if n == 100:
        plt.xticks(range(x.min(), x.max()+1, 2))
    else:
        plt.xticks(x)
    plt.plot(x, binom.pmf(x, n, p), ':or', label="PMF")
    plt.legend()

n = widgets.SelectionSlider(options=list(range(1,11))+[10, 50, 100], value=1,
    description='Número de monedas a lanzar',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

p = widgets.SelectionSlider(value=0.5, options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                            description='Probabilidad (cara)',
                            layout=widgets.Layout(width='70%'))
p.style = {'description_width': '300px'}

s = widgets.Dropdown(options=[10, 100, 1000, 10000, 100000], value=100,
    description='Número de lanzamientos',
    layout=widgets.Layout(width='70%'))
s.style = {'description_width': '300px'}

ui = widgets.VBox(children=[n, p, s])

out = widgets.interactive_output(coin, {'n': n, 's': s, 'p': p})

display(ui, out)

Output()

### Ley de los grandes números

- Supongamos que tenemos una v.a. i.i.d. $X_1 , X_2 , X_3 , \dots$
con media finita $\mu$ y varianza finita $\sigma^2$.
Para todos los enteros positivos $n$, sea
$$
\bar{X}_n=\frac{X_1 + \cdots + X_n}{n}
$$
sea la media muestral de $X_1$ a $X_n$.
La media muestral es en sí misma una v.a., con media
$\mu$ y varianza $\sigma^2/n$:
$$
E(\bar{X}_n)=\frac{1}{n}E(X_1+\cdots+X_n)=
\frac{1}{n}(E(X_1)+\cdots+E(X_n))=\mu,
$$
$$
Var(\bar{X}_n)=\frac{1}{n^2}Var(X_1+\cdots+X_n)=
\frac{1}{n^2}(Var(X_1)+\cdots+Var(X_n))=\frac{\sigma^2}{n} \rightarrow 0.
$$

- La ley de los grandes números dice que a medida que el tamaño de la muestra $n$ crece, el valor esperado de la media muestral $E[\bar{X}]_n$ converge a la verdadera media $\mu$, y la varianza $Var[\bar{X}]_n$ converge a cero.

In [7]:
def lln(n, dist, a):
    simulations = 1000
    if dist=="Uniforme":
        X_bar = np.random.random(size=(simulations,n)).mean(axis=1)
        mu, sigma2 = 1/2, 1/12  # mu, sigma of the population distribution
        ll, ul = 0, 1
    if dist=="Exponencial":
        X_bar = expon.rvs(scale=2, size=(simulations,n)).mean(axis=1)
        mu, sigma2 = 2, 4  # mu, sigma of the population distribution
        ll, ul = 0, 10
    fig, ax = plt.subplots(2, 1, figsize=(7,7), gridspec_kw={'height_ratios': [3, 1]})
    fig.tight_layout()
    ax[0].plot(X_bar, range(simulations), '.', label=r"$\bar{X}$", alpha=a)
    ax[0].vlines(mu, 0, simulations, linewidth=2, color="red", label=r"$\mu$")
    ax[0].set_xlim(ll, ul)
    ax[0].legend(loc=1)

    ax[1].hist(X_bar, density=True)
    ax[1].set_xlim(ll, ul)
    ax[1].set_xlabel("X")
    
n = widgets.SelectionSlider(options=[1, 2, 5, 10, 30, 100, 1000, 10000, 50000], value=1,
    description='Tamaño de muestra',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

dist = widgets.Dropdown(options=["Uniforme", "Exponencial"], value="Uniforme",
                       description='Distribución',
    layout=widgets.Layout(width='70%'))
dist.style = {'description_width': '300px'}

a = widgets.SelectionSlider(options=[0.2, 0.5, 0.8, 1], value=0.2,
    description='Transparencia',
    layout=widgets.Layout(width='70%'))
a.style = {'description_width': '300px'}

ui = widgets.VBox(children=[n, dist, a])

out = widgets.interactive_output(lln, {'n': n, 'dist': dist, 'a': a});

display(ui, out);

Output()

### Teorema del límte central

- Elija el tamaño de muestra $n$ para el cual quiere calcular la media muestral $\bar{X}$. 
  - Este proceso se repite muchas veces y se grafica un histograma de la media muestral.
- Se grafica la PDF de la distribución normal $\bar{X} \sim N(\mu, \sigma^2/\sqrt{n})$.
  - El teorema del límite central nos dice que $\bar{X}$ sigue dicha distribución para un tamaño lo suficientemente grande de $n$. 
- El muestreo proviene de 2 distribuciones:
  - Uniforme $X\sim U(a=0, b=1)$ con: $\mu=1/2,\ \sigma^2 = 1/12$.
  - Exponencial $X\sim Expon(\lambda=2)$ con: $\mu=2,\ \sigma^2 = 4$.

In [8]:
def clt(n, dist):
    simulations = 10000
    if dist=="Uniforme":
        X_bar = np.random.random(size=(simulations,n)).mean(axis=1)
        mu, sigma2 = 1/2, 1/12  # mu, sigma of the population distribution
    if dist=="Exponencial":
        X_bar = expon.rvs(scale=2, size=(simulations,n)).mean(axis=1)
        mu, sigma2 = 2, 4  # mu, sigma of the population distribution
    plt.figure(figsize=(8,6))
    plt.hist(X_bar, bins=40, density=True, edgecolor="seagreen", label=r"$\bar{X}$")
    xl, xu = X_bar.min(), X_bar.max()
    x = np.linspace(xl, xu, 500)
    plt.plot(x, norm.pdf(x, loc=mu, scale=np.sqrt(sigma2/n)), 
             linewidth=2, color="crimson", label=r"$N(\mu, \sigma^2/\sqrt{n})$")
    plt.legend()
    

n = widgets.Dropdown(options=[1, 2, 5, 10, 30, 100], value=1,
    description='Tamaño de muestra',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

dist = widgets.Dropdown(options=["Uniforme", "Exponencial"], value="Uniforme",
                       description='Distribución',
    layout=widgets.Layout(width='70%'))
dist.style = {'description_width': '300px'}

ui = widgets.VBox(children=[n, dist])

out = widgets.interactive_output(clt, {'n': n, 'dist': dist})

display(ui, out)

Output()

### Exactitud y precisión de un estimador $\theta_j$


<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/Illustration-of-the-precision-and-the-accuracy-of-an-estimator.png" alt="drawing" width="400"/>

In [9]:
def throw_darts(n, e, p):
    smx, smy = [np.random.choice([-1, 1], size=2)][0]
    exactitud = {'Alta': (0, 0), 'Media': (0.5, 0.5), 'Baja': (2, 2)}
    precision = {'Alta': (0.1, 0.1), 'Media': (0.25, 0.25), 'Baja': (1, 1)}
    mx, my = exactitud[e]
    sx, sy = precision[p]
    mx *= smx; my *= smy
    dart_x = norm.rvs(mx, sx, size=n)
    dart_y = norm.rvs(my, sy, size=n)

    plt.figure(figsize=(8,8))

    plt.plot(0, 0, 'ok', label="Target")
    plt.plot(dart_x, dart_y, 'x')

    circle1 = plt.Circle((0,0),1,color='b', fill=False)
    plt.gcf().gca().add_artist(circle1)
    circle1 = plt.Circle((0,0),2,color='g', fill=False)
    plt.gcf().gca().add_artist(circle1)
    circle1 = plt.Circle((0,0),3,color='r', fill=False)
    plt.gcf().gca().add_artist(circle1)

    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    

n = widgets.Dropdown(options=[10, 20, 30, 50, 100], value=10,
    description='Tamaño de muestra',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

e = widgets.Dropdown(options=["Alta", "Media", "Baja"], value="Alta",
                       description='Exactitud',
    layout=widgets.Layout(width='70%'))
e.style = {'description_width': '300px'}

p = widgets.Dropdown(options=["Alta", "Media", "Baja"], value="Alta",
                       description='Precisión',
    layout=widgets.Layout(width='70%'))
p.style = {'description_width': '300px'}

ui = widgets.VBox(children=[n, e, p])

out = widgets.interactive_output(throw_darts, {'n': n, 'e': e, 'p': p})

display(ui, out)

Output()

### Exactitud y precisión

- Experimenta por tu cuenta

In [10]:
def throw_darts(mx, my, sx, sy, n=20):
    dart_x = norm.rvs(mx, sx, size=n)
    dart_y = norm.rvs(my, sy, size=n)

    plt.figure(figsize=(8,8))

    plt.plot(0, 0, 'ok', label="Target")
    plt.plot(dart_x, dart_y, 'x')

    circle1 = plt.Circle((0,0),1,color='b', fill=False)
    plt.gcf().gca().add_artist(circle1)
    circle1 = plt.Circle((0,0),2,color='g', fill=False)
    plt.gcf().gca().add_artist(circle1)
    circle1 = plt.Circle((0,0),3,color='r', fill=False)
    plt.gcf().gca().add_artist(circle1)

    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    

mx = widgets.SelectionSlider(value=0, options=[-2, -1, -0.5, 0, 0.5, 1, 2],
                            description='Media X',
                            layout=widgets.Layout(width='70%'))
mx.style = {'description_width': '300px'}

my = widgets.SelectionSlider(value=0, options=[-2, -1, -0.5, 0, 0.5, 1, 2],
                            description='Media Y',
                            layout=widgets.Layout(width='70%'))
my.style = {'description_width': '300px'}

sx = widgets.SelectionSlider(value=0.1, options=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.0],
                            description='Desviación X',
                            layout=widgets.Layout(width='70%'))
sx.style = {'description_width': '300px'}

sy = widgets.SelectionSlider(value=0.1, options=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.0],
                            description='Desviación Y',
                            layout=widgets.Layout(width='70%'))
sy.style = {'description_width': '300px'}

ui = widgets.VBox(children=[mx, my, sx, sy])

out = widgets.interactive_output(throw_darts, {'mx': mx, 'my': my, 'sx': sx, 'sy': sy})

display(ui, out)

Output()

### Intervalo t de confianza para la media

- Vamos a realizar un muestreo de una distribución normal estándar $X\sim N(0,1)$.
- Construiremos un intervalo t para la media. 
- Vea como cambia el ancho del intervalo de acuerdo al nivel de confianza y al tamaño de muestra. 

In [11]:
def interval(nc, n, size=100):
    alpha = 1 - nc
    df = n - 1  # grados de libertad para t

    # Estamos muestreando la distribución uniforme estándar
    mu = 0  # mu: media poblacional

    U = norm.rvs(size=(size,n))  # uniform rvs
    Um = np.mean(U, axis=1)  # media muestral 
    Us = np.std(U, ddof=1, axis=1)  # varianza muestral
    se = Us / np.sqrt(n)  # error estándar

    ICL, ICU = t.interval(1-alpha, df, loc=Um, scale=se)  # Intervalo de confianza 

    CI = np.vstack([ICL, ICU]).transpose()  # unimos ICL, ICU en 2 columnas
    # comprobamos que mu está en el intervalo
    # para esto debe cumplirse que limite inferior < mu y límite superior mayor que mu
    # np.logical_and(CI[:,0] < mu, CI[:,1] > mu)
    # ubicamos los índices donde se cumple la condición anterior 
    # np.where(condicion)[]
    cont = np.where(np.logical_and(CI[:,0] < mu, CI[:,1] > mu))[0]

    plt.figure(figsize=(8,8))

    # Graficamos cada intervalo como una línea horizontal
    # Si el intervalo contiene la media poblacinal mu es negro
    # rojo en caso contrario
    for pos, limits in enumerate(CI):
        if pos in cont:  # está dentro del intervalo
            col = 'k'  # color negro
        else:  # está fuera del intervalo
            col = 'r'  # color rojo
        plt.hlines(pos+1, limits[0], limits[1], colors=col)  # línea horizontal
        plt.plot(Um[pos], pos+1, '.', c=col)

    plt.vlines(mu, 0, size+1, colors='b', label=r"$\mu$")
    plt.legend()
    plt.xlim(-1.5, 1.5)

    # Contemos cuantas veces el intervalo contiene la media poblacional
    # Si el intervalo (lineal horizontal) contiene la media, este se intersecta con la línea vertical azul
    l.value = "La media poblacional está en el intervalo "+str(np.shape(cont)[0]/size*100)+"% de las veces."
    
    
nc = widgets.SelectionSlider(value=0.9, options=[0.68, 0.9, 0.95, 0.99],
                            description='Nivel de confianza',
                            layout=widgets.Layout(width='70%'))
nc.style = {'description_width': '300px'}

n = widgets.Dropdown(options=[10, 30, 100, 500, 1000], value=30,
    description='Tamaño de muestra',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

l = widgets.Label(" ")

ui = widgets.VBox(children=[nc, n, l])

out = widgets.interactive_output(interval, {'nc': nc, 'n': n})

display(ui, out)

Output()

### Error de Tipo I ($\alpha$), Tipo II ($\beta$) y poder de la prueba (power=$1-\beta$)


- Suponga que realizará una muestra aleatoria de tamaño $n$ de una población distribuida $X \sim N(\mu, sigma^2)$, donde $\sigma=21$ y $\mu$ es desconocida.

- Vamos a probar:

$$H_0: \mu=50$$
$$H_1: \mu < 50$$

- Con un nivel de significancia de $\alpha=0.09$. 

- Si en realidad $\mu = \mu_1 =40$, cual es la $P(\text{Error de tipo II})$. 

- Cambie el nivel de significancia, valores de $\mu_0, \mu_1$ y el tamaño de muestra $n$ y observe el cambio en los valores de $\beta$, power=$1-\beta$.

In [12]:
def errors(n, mu1, alpha):
    #n = 36
    mu = 50  # H_A, hipótesis nula
    s = 21  # desviacioón poblacional conocida sigma
    se = s / np.sqrt(n)

    # Graficamos la distribución X
    x = np.linspace(mu-4*se, mu+4*se, 200)  # x a +- 4 desviaciones de la media
    pdfx = norm.pdf(x, mu, se)  # zx = N(0,1)
    plt.figure(figsize=(8,6))
    plt.plot(x, pdfx)

    # Graficamos punto crítico y sombreamos región crítica
    # cola superior de acuerdo a la hipótesis alternativa H1
    #alpha = 0.09  # nivel de significancia
    xc = norm.ppf(alpha, mu, se)  # punto crítico, cola inferior z(alpha)

    x_alpha = np.linspace(mu-4*se, xc)  # cola inferior
    plt.fill_between(x_alpha, norm.pdf(x_alpha, mu, se), 
                     color="orange", alpha=.5, hatch='/',
                     label=r"$\alpha$")
    plt.vlines(mu, 0, norm.pdf(mu, mu, se), 
               linestyles="dashed", color="cornflowerblue", label=r"$\mu=$"+str(mu))

    #mu1 = 40  # H_B, realidad

    # Graficamos la distribución X
    x = np.linspace(mu1-4*se, mu1+4*se, 200)  # x a +- 4 desviaciones de la media
    pdfx = norm.pdf(x, mu1, se)  # zx = N(0,1)
    plt.plot(x, pdfx, 'g')
    plt.vlines(xc, 0, norm.pdf(xc, mu1, se), color="g")
    x_beta = np.linspace(xc, mu1+4*se)  # beta
    plt.fill_between(x_beta, norm.pdf(x_beta, mu1, se), 
                     color="green", alpha=.3,
                     label=r"$\beta$")
    plt.vlines(mu1, 0, norm.pdf(mu1, mu1, se), 
               linestyles="dashed", color="green", label=r"$\mu=$"+str(mu1))

    x_power = np.linspace(mu1-4*se, xc)  # power
    plt.fill_between(x_power, norm.pdf(x_power, mu1, se), 
                     color="cyan", alpha=.2, hatch='o',
                     label=r"$power$")

    plt.xlabel(r'$\bar{X}$')
    l = plt.legend()
    beta = 1-norm.cdf(xc, mu1, se) 
    print({"Valor crítico": np.round(xc, 4), 
           "beta":  np.round(beta, 4), "power":  np.round(1-beta, 4)})

alpha = widgets.SelectionSlider(value=0.09, options=[0.01, 0.05, 0.09, 0.1, 0.2, 0.32],
                            description=r'Nivel de significancia ($\alpha$)',
                            layout=widgets.Layout(width='70%'))
alpha.style = {'description_width': '300px'}

n = widgets.Dropdown(options=[10, 25, 30, 36, 100, 500], value=36,
    description='Tamaño de muestra',
    layout=widgets.Layout(width='70%'))
n.style = {'description_width': '300px'}

mu1 = widgets.SelectionSlider(options=[35, 40, 43, 45, 46, 47, 48, 49, 50], value=40,
    description=r'Media verdadera $\mu_1$',
    layout=widgets.Layout(width='70%'))
mu1.style = {'description_width': '300px'}

ui = widgets.VBox(children=[n, mu1, alpha])

out = widgets.interactive_output(errors, {'n': n, 'mu1': mu1, 'alpha': alpha})

display(ui, out)

Output()

### Bono: Análisis exploratorio

- Para realizar un análisis exploratorio de datos (EDA), así como para visualizar efectivamente un conjunto de datos es necesario tener interactividad. 
  - La interactividad permite flexibilidad al usuario para elegir las variables a explorar. 

- Para la interactividad usaremos **ipywidgets** junto con **seaborn** para visualizar los datos. 

  - Vea el siguiente [enlace](https://ppeng08.medium.com/interactive-visualization-for-exploratory-data-analysis-in-jupyter-notebook-adc826e1e76a).

In [13]:
tips = sns.load_dataset("tips")
@ipywidgets.interact
def plot(col=tips.select_dtypes(include = 'category').columns):          # categorical univariate plot
    plt.figure(figsize=(8,6))
    sns.countplot(y=col, data=tips);                                     # y indicates horizontal plot

interactive(children=(Dropdown(description='col', options=('sex', 'smoker', 'day', 'time'), value='sex'), Outp…

In [14]:
@ipywidgets.interact
def plot(col_x=tips.select_dtypes(include = 'float').columns, 
         hue=tips.select_dtypes(include = 'category').columns,
         element=["bars", "step", "poly"]
        ):
    sns.displot(data=tips, x=col_x, hue=hue, element=element);

interactive(children=(Dropdown(description='col_x', options=('total_bill', 'tip'), value='total_bill'), Dropdo…

In [15]:
@ipywidgets.interact
def plot(var_x=tips.select_dtypes(include = 'category').columns, 
         var_y=tips.select_dtypes(include = 'float').columns,
         hue=tips.select_dtypes(include = 'category').columns,
         tplot=["boxplot", "lineplot"]
        ):         
    if tplot == "boxplot":
        sns.boxplot(data=tips, x=var_x, y=var_y, hue=hue);
    else:
        sns.lineplot(data=tips, x=var_x, y=var_y, hue=hue, 
                     err_style="bars", ci=68, estimator='mean')

interactive(children=(Dropdown(description='var_x', options=('sex', 'smoker', 'day', 'time'), value='sex'), Dr…

In [16]:
@ipywidgets.interact
def plot(col_x=tips.select_dtypes(include = 'float').columns, 
         col_y=tips.select_dtypes(include = 'float').columns, 
         hue=tips.select_dtypes(include = 'category').columns,
         size=tips.select_dtypes(include = 'category').columns
        ): 
    sns.scatterplot(data=tips, x=col_x, y=col_y, hue=hue, size=size)

interactive(children=(Dropdown(description='col_x', options=('total_bill', 'tip'), value='total_bill'), Dropdo…

In [17]:
@ipywidgets.interact
def plot(col=tips.select_dtypes(include = 'category').columns,
         hue=tips.select_dtypes(include = 'category').columns,
        ):          # categorical univariate plot
    sns.countplot(x=col, hue=hue, data=tips);

interactive(children=(Dropdown(description='col', options=('sex', 'smoker', 'day', 'time'), value='sex'), Drop…

### Bono: Análisis exploratorio

- Replicamos el ejercicio anterior usando **plotly** en lugar de seaborn. 
- Plotly crea gráficos interactivos a diferencia de los gráficos estáticos de seaborn. 

In [18]:
tips = sns.load_dataset("tips")
@ipywidgets.interact
def plot(col=tips.select_dtypes(include = 'category').columns):
    df = tips.groupby(by=[col]).size().reset_index(name="counts")
    fig = px.bar(data_frame=df, x=col, y="counts", color=col)
    fig.show()

interactive(children=(Dropdown(description='col', options=('sex', 'smoker', 'day', 'time'), value='sex'), Outp…

In [19]:
@ipywidgets.interact
def plot(col_x=tips.select_dtypes(include = 'float').columns, 
         hue=tips.select_dtypes(include = 'category').columns,
         marginal=["box", "violin", "rug"]
        ):
    fig = px.histogram(data_frame=tips, x=col_x, color=hue, marginal=marginal)
    fig.show()

interactive(children=(Dropdown(description='col_x', options=('total_bill', 'tip'), value='total_bill'), Dropdo…

In [20]:
@ipywidgets.interact
def plot(col_x=tips.select_dtypes(include = 'float').columns, 
         hue=tips.select_dtypes(include = 'category').columns
        ):
    group_labels = tips[hue].unique()
    groups = []
    for gl in group_labels:
        groups += [tips[col_x][tips[hue]==gl]]
    fig = ff.create_distplot(groups, group_labels)
    fig.show()

interactive(children=(Dropdown(description='col_x', options=('total_bill', 'tip'), value='total_bill'), Dropdo…

In [21]:
@ipywidgets.interact
def plot(var_x=tips.select_dtypes(include = 'category').columns, 
         var_y=tips.select_dtypes(include = 'float').columns,
         hue=tips.select_dtypes(include = 'category').columns,
         points=["outliers", "all"], faceted = False
        ):
    if faceted: facet_ = hue
    else: facet_ = None
    fig = px.box(tips, x=var_x, y=var_y, color=hue, facet_col=facet_, points=points)
    fig.show()

interactive(children=(Dropdown(description='var_x', options=('sex', 'smoker', 'day', 'time'), value='sex'), Dr…

In [22]:
@ipywidgets.interact
def plot(var_x=tips.select_dtypes(include = 'float').columns, 
         var_y=tips.select_dtypes(include = 'float').columns, 
         hue=tips.select_dtypes(include = 'category').columns,
         marker=tips.select_dtypes(include = 'category').columns,
         faceted = False
        ):
    if faceted: facet_ = hue
    else: facet_ = None
    fig = px.scatter(data_frame=tips, x=var_x, y=var_y, color=hue, symbol=marker, facet_col=facet_)
    fig.show()

interactive(children=(Dropdown(description='var_x', options=('total_bill', 'tip'), value='total_bill'), Dropdo…

In [23]:
from plotly.offline import iplot

@ipywidgets.interact
def plot(col=tips.select_dtypes(include = 'category').columns,
         hue=tips.select_dtypes(include = 'category').columns,
        ):          # categorical univariate plot
    if col==hue:
        df = tips.groupby(by=[col]).size().reset_index(name="counts")
    else:
        df = tips.groupby(by=[col, hue]).size().reset_index(name="counts")
    fig = px.bar(data_frame=df, x=col, y="counts", color=hue, barmode="group")
    fig.show()

interactive(children=(Dropdown(description='col', options=('sex', 'smoker', 'day', 'time'), value='sex'), Drop…